In [16]:
# Read the JSON file downloaded from the free Yelp dataset
# Goal is to use Spark to clean the data to just list the restaurants in California

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Creating SparkSession #
# spark = SparkSession.builder.remote("sc://192.168.4.107:15002".appName("JSONImport").getOrCreate()
spark = SparkSession.builder.appName("JSONImport").getOrCreate()

# Reading JSON file #
df = spark.read.json("work/yelp_academic_dataset_business.json")
# df.show()
# df.printSchema()

# Creating a new DataFrame with just the columns needed #
# Only need columns 'address', 'categories', 'city', 'latitude', 'longitude', 'name', 'postal_code', 'state'
# Dropping all other columns
col_to_remove = ["attributes", "business_id", "hours", "is_open", "review_count", "stars"]
df_col_drop = df.drop(*col_to_remove)
#df_col_drop.show()

# Explode out categories #
df_cat_explode = df_col_drop.withColumn("Indiv_Cat", f.explode(f.split(f.col("categories"),","))).withColumn("Indiv_Cat", \
                                                                                                                  f.trim(f.col("Indiv_Cat")))
#df_cat_explode.show()

# Create df_restaurants with just a list of Restaurants #
df_restaurants = df_cat_explode.filter(df_cat_explode["Indiv_Cat"] == "Restaurants")
#df_restaurants.show()

# Create df_restaurants_CA with just restaurants in CA #
df_restaurants_CA = df_restaurants.filter(df_restaurants["state"] == "CA")
#df_restaurants_CA.show()

# Grab a list of distinct cities to compare to the API #
city_list = [row.city for row in df_restaurants_CA.select("city").distinct().collect()]
#city_list = df_restaurants_CA.select("city").distinct().collect()
print (city_list)



spark.stop()

['Isla Vista', 'Carpinteria', 'Goleta', 'Santa Barbara', 'Montecito', 'Summerland', 'Truckee', 'Santa  Barbara']
